In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [ ]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DoubleType
from pyspark.ml.linalg import Vectors

In [ ]:
spark = SparkSession.builder.appName("DecisionTreeRegression").getOrCreate()

In [ ]:
schema = StructType([
        StructField('Rank', IntegerType(), True),
        StructField('Name', StringType(), True),
        StructField('Platform', StringType(), True),
        StructField('Year', IntegerType(), True),
    StructField('Genre', StringType(), True),
    StructField('Publisher', StringType(), True),
    StructField('NA_Sales', DoubleType(), True),
    StructField('EU_Sales', DoubleType(), True),
    StructField('JP_Sales', DoubleType(), True),
    StructField('Other_Sales', DoubleType(), True),
    StructField('Global_Sales', DoubleType(), True)
        ])

In [ ]:
df = spark.read.csv('vgsales.csv',inferSchema=True,header='true')

In [ ]:
data=df.rdd.map(lambda x:(Vectors.dense(x[6],x[7],x[8]), x[9])).toDF(["features", "label"])

In [ ]:
featureIndexer =VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4,handleInvalid='skip').fit(data)

In [ ]:
(trainingData, testData) = data.randomSplit([0.7, 0.3])

In [ ]:
dt = DecisionTreeRegressor(featuresCol="indexedFeatures")

In [ ]:
pipeline = Pipeline(stages=[featureIndexer, dt])

In [ ]:
model = pipeline.fit(trainingData)

In [ ]:
predictions = model.transform(testData)

In [ ]:
predictions.select("prediction", "label", "features").show(5)

In [ ]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

In [ ]:
treeModel = model.stages[1]

In [ ]:
print(treeModel)